# Unit-Aware Plotting with SciTeX

This notebook demonstrates the new unit-aware plotting capabilities in SciTeX, which ensure scientific validity by tracking and converting physical units automatically.

In [ ]:
import numpy as np
import scitex as stx
from scitex.units import Q, Units

## 1. Basic Unit-Aware Plotting

The simplest way to use unit-aware plotting is to specify units when plotting:

In [ ]:
# Generate data
time_ms = np.linspace(0, 100, 1000)
frequency = 50  # Hz
signal_mv = 5 * np.sin(2 * np.pi * frequency * time_ms / 1000)

# Create unit-aware plot
fig, ax = stx.plt.subplots(figsize=(10, 6))

# Plot with units - units are automatically added to labels
ax.plot_with_units(time_ms, signal_mv, x_unit='ms', y_unit='mV', 
                   color='blue', linewidth=2, label='50 Hz Signal')

ax.set_xlabel('Time')  # Unit [ms] is added automatically
ax.set_ylabel('Voltage')  # Unit [mV] is added automatically
ax.set_title('Unit-Aware Plotting Example')
ax.grid(True, alpha=0.3)
ax.legend()

stx.io.save(fig, 'unit_aware_basic_example')

## 2. Unit Conversion

Convert between different units easily:

In [ ]:
# Create figure with two subplots
fig, (ax1, ax2) = stx.plt.subplots(2, 1, figsize=(10, 8))

# Plot 1: Original units (milliseconds, millivolts)
ax1.plot_with_units(time_ms, signal_mv, x_unit='ms', y_unit='mV',
                    color='blue', linewidth=2)
ax1.set_xlabel('Time')
ax1.set_ylabel('Voltage')
ax1.set_title('Original Units')
ax1.grid(True, alpha=0.3)

# Plot 2: Same data, different display units
ax2.plot_with_units(time_ms, signal_mv, x_unit='ms', y_unit='mV',
                    color='red', linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Voltage')

# Convert to SI units
ax2.convert_x_units('s')  # Convert ms to s
ax2.convert_y_units('V')  # Convert mV to V

ax2.set_title('Converted to SI Units')
ax2.grid(True, alpha=0.3)

fig.tight_layout()
stx.io.save(fig, 'unit_conversion_example')

## 3. Using Quantity Objects

SciTeX integrates with the units module to work with Quantity objects directly:

In [ ]:
# Create quantities with units
time = Q(np.linspace(0, 0.1, 1000), Units.s)
frequency = Q(50, Units.Hz)
amplitude = Q(230, Units.V)

# Calculate signal (units are tracked automatically)
omega = 2 * np.pi * frequency.value
signal_values = amplitude.value * np.sin(omega * time.value)
signal = Q(signal_values, Units.V)

# Plot quantities directly
fig, ax = stx.plt.subplots(figsize=(10, 6))

# Units are automatically detected from Quantity objects
ax.plot_with_units(time, signal, color='green', linewidth=2)

ax.set_xlabel('Time')  # Automatically shows [s]
ax.set_ylabel('Voltage')  # Automatically shows [V]
ax.set_title(f'AC Mains Voltage ({frequency.value} {frequency.unit.symbol})')
ax.grid(True, alpha=0.3)

# Add RMS voltage annotation
rms_voltage = amplitude.value / np.sqrt(2)
ax.axhline(y=rms_voltage, color='red', linestyle='--', alpha=0.5, label=f'RMS: {rms_voltage:.1f} V')
ax.axhline(y=-rms_voltage, color='red', linestyle='--', alpha=0.5)
ax.legend()

stx.io.save(fig, 'quantity_plotting_example')

## 4. Unit Validation

SciTeX can validate unit compatibility to prevent errors:

In [ ]:
# Create figure
fig, ax = stx.plt.subplots(figsize=(10, 6))

# Set expected units for axes
ax.set_x_unit('ms')
ax.set_y_unit('mV')

# Plot data with correct units
time_ms = np.linspace(0, 100, 100)
voltage_mv = 10 * np.sin(2 * np.pi * 10 * time_ms / 1000) + np.random.randn(100)

ax.plot_with_units(time_ms, voltage_mv, color='blue', label='Noisy Signal')

# Unit validation can be toggled
ax.set_unit_validation(False)  # Disable for flexibility

# Now we can plot anything (use with caution!)
ax.plot_with_units(time_ms, time_ms**2 / 1000, color='red', 
                   linestyle='--', label='Quadratic (no units)')

ax.set_xlabel('Time')
ax.set_ylabel('Voltage')
ax.set_title('Unit Validation Example')
ax.legend()
ax.grid(True, alpha=0.3)

stx.io.save(fig, 'unit_validation_example')

## 5. Scientific Example: RC Circuit Response

Let's use unit-aware plotting for a real scientific application:

In [ ]:
# RC circuit parameters
R = Q(1000, Units.ohm)  # 1 kΩ resistor
C = Q(1e-6, Units.farad)  # 1 μF capacitor
V_source = Q(5, Units.V)  # 5V source

# Time constant
tau = R.value * C.value  # seconds
print(f"Time constant τ = {tau*1000:.1f} ms")

# Generate time array
t = np.linspace(0, 5*tau, 1000)
t_ms = t * 1000  # Convert to milliseconds for plotting

# Calculate voltages
V_capacitor = V_source.value * (1 - np.exp(-t/tau))
V_resistor = V_source.value * np.exp(-t/tau)

# Create figure
fig, (ax1, ax2) = stx.plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Plot capacitor voltage
ax1.plot_with_units(t_ms, V_capacitor, x_unit='ms', y_unit='V',
                    color='blue', linewidth=2, label='Capacitor')
ax1.axhline(y=V_source.value * 0.632, color='red', linestyle='--', alpha=0.5)
ax1.text(tau*1000, V_source.value * 0.632, f' 63.2% at τ = {tau*1000:.1f} ms',
         va='bottom', ha='left', color='red')
ax1.set_ylabel('Voltage')
ax1.set_title('RC Circuit Step Response')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot current (V_R / R)
current_mA = (V_resistor / R.value) * 1000  # Convert to mA
ax2.plot_with_units(t_ms, current_mA, x_unit='ms', y_unit='mA',
                    color='green', linewidth=2, label='Current')
ax2.set_xlabel('Time')
ax2.set_ylabel('Current')
ax2.grid(True, alpha=0.3)
ax2.legend()

fig.tight_layout()
stx.io.save(fig, 'rc_circuit_example')

## Summary

The unit-aware plotting features in SciTeX provide:

1. **Automatic unit tracking** - Units are displayed in axis labels
2. **Unit conversion** - Easy conversion between different unit systems
3. **Integration with Quantity objects** - Seamless plotting of unit-aware data
4. **Unit validation** - Optional checking for unit compatibility
5. **Scientific validity** - Ensures plots are physically meaningful

This makes SciTeX ideal for scientific computing where unit consistency is crucial.